# Testing code using unittest module in Python

This will serve as a __template__ that hopefully everyone can use to test various parts of this platform and will be a brief tutorial on unittest module in python. This is based on some tutorials as well as some other exercises that I've tested using unittest. This is very important for big projects as updating a piece of code can break other parts of the code base and these test cases will catch that. 

For those interested, I recommend this [video](https://www.youtube.com/watch?v=6tNS--WetLI) to learn the basics of [unittest](https://docs.python.org/3/library/unittest.html). 

Unittest is native to python therefore, you can simply import without downloading any extra packages. 

In __spiketorch/examples/testing__, you will find __demo.py__ and __demo_tests.py__ which are the files being explained here. I will be using snippits of code from there and explaining their purpose here.  

Here are the [unittest assert methods](https://docs.python.org/3/library/unittest.html#unittest.TestCase.debug) that are useful in comparing results at the end of every test function. 

__Disclaimer:__ Code is not runnable since unittest.main() runs the tests on its own. The examples are simply for visual aid. If there is confusion at any point, please visit the files mentioned above as the examples are commented. 

## Imports

First, import unittest and the rest of the imports are used for the examples below.

In [1]:
import unittest
import numpy as np
import unittest
import sys
%load_ext autoreload
%autoreload 2

Create a class (giving descriptive title to classes helps) and inherit __unittest.TestCase__. This will allow us to use different testing capabilities within that class. 

__Important:__ Every test will be in the form of a function and must always start with "test" otherwise unittest.main() will not run the test function you created. A standard naming convention for test functions is __"test_whatever_you_are_testing()"__. Since every test function is a method inside of the class, it's first argument must be __self__ as always.


The number of tests that will run is equal to the number of functions beginning with "test". It is __NOT__ the number of times you call an assert method inside of a test function.

## Test structure conventions

In [ ]:
class TestClass(unittest.TestCase): # Good name for class
    def test_addition(self): # Valid name for test
        pass
    
    def addition_test(self): # incorrect name, names of tests MUST start with "test" 
        pass

A good way to structure all the tests would be to name one function "test" that calls all of the other test functions inside it. And then, you can write all of the test functions below it. Example shown below. 

In [ ]:
class TestClass(unittest.TestCase):
    
    def test(self):
        print("*********** Testing ***********")
        self._test1(function_name, test_name)
        self._test2(function_name, test_name)
        ...
        
    def _test1(self, function_name, test_name):
        print("---- %s ----"%test_name)
        ...
        ...
        ...

Since the rest of the functions start with '\_' , they will not be called by _unittest.main()_. The "check_load" function is a general way of checking whether the file and the function you want to call exist or not. They will throw errors if file or function is not found. 

__Important:__ If an updated piece of code produces an error, it is important to then update the types of tests that are being conducted to also incorporate a test that can catch the particular error that was just produced by the code. 

## Ways of importing file and function

I have implemented check_load function that checks the existence of desired module and function and will throw error otherwise. It's an elegant way of implementing it (shown below). There will be an example below that uses this function to load file and function which should clear up points of confusion (if there are any).

In [ ]:
def check_load(filename,fun_name):
    """
    A wrapper function to load modules and functions to test. Produces error 
    if file or function is not found.
    
    Inputs: -> Filename that contains the functions we want to call without the ".py" extensions
            -> Function name that is inside filename that we want to call
            
    Output: -> Desired function is returned
    """
    try:
        #Importing the file source code
        user_module = __import__(filename) 
    except ModuleNotFoundError: 
        print("%s FILE NOT FOUND"%filename)
        return None
    
    """
     Everything in python is an object and importing this way will make it so that 
     user_module has a __dict__ which is a dictionary that contains all of the 
     relevant information about filename in it including the desired function
    """
    if hasattr(user_module,fun_name) and callable(user_module.__dict__[fun_name]):
        print("%s was succesfully loaded"%fun_name)
        return user_module.__dict__[fun_name] #return the function pointer for usage
    else: #did not find the function
        print("%s is not defined"%fun_name)
        return None

In [ ]:
from demo import gauss_jordan #a simple import statement will work as well

Having __unittest.main()__ this way allows for an interface between the IDE and command line. Normally, you would have to run command on the cmd using -m flag. For example, to run my test file I would run: "python -m unittest demo_tests.py"

In [ ]:
if __name__=="__main__":
    unittest.main() # used primarily for command line

## More useful functions 

If in every test case you see youself performing the same line of code, that can be reduced for a cleaner set of test functions. There are two methods that I will mention, [setUp()](https://docs.python.org/3.4/library/unittest.html#unittest.TestCase.setUp) which the testing framework will automatically call for every single test we run and [tearDown()](https://docs.python.org/3.4/library/unittest.html#unittest.TestCase.tearDown) method that tidies up after the test method has been run. This means these methods will be called for every single test FUNCTION that we create. 

__Important:__ Since we are no longer creating instances of variables in every test function, the variables we create in setUp() must use __self__ to be part of the class so that they can be used in the rest of the test functions. 

Another important thing here is that in order to have one piece of code run __once before__ any tests and __once after__ all the tests have finishes, we need to use class methods. Here is another [video](https://www.youtube.com/watch?v=rq8cL2XMM5M) that explains class methods which might be helpful. Otherwise, the example I'm showing should suffice.  

In [ ]:
class TestClass(unittest.TestCase): 
    def setUp(self):
        """
        Place repeated code here to run for every single test function
        """
        print('setUp')
    
    def tearDown(self):
        """
        This will run after every test function finishes executing
        """
        print('tearDown\n')

    @classmethod
    def setUpClass(cls):
        print('setUpClass\n')
    
    @classmethod
    def tearDownClass(cls):
        print('tearDownClass')
        
    #TEST FUNCTIONS
    def test_1(self):
        print('test1')
    
    def test_2(self):
        print('test2')

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
        

[setUpClass](https://docs.python.org/3/library/unittest.html#unittest.TestCase.setUpClass) will be very useful in our cases since we can initialize the entire network once. An example given below.

We could introduce functions that completely reset the entire network which could be used inside tearDown() for a more efficient test suite.

In [ ]:
@classmethod
    def setUpClass(cls):
        timesteps = 500
        n_inpt = 100
        n_excitatory = 100

        p_fire = 0.05
        
        network = Network(dt=1)  # timestep of 1ms

        # Neuron groups.
        network.add_group(InputGroup(n_inpt), 'input')
        network.add_group(LIFGroup(n_excitatory), 'population')

        # Synaptic connections.
        w = torch.randn(n_inpt, n_excitatory)
        network.add_synapses(Synapses(network.groups['input'], network.groups['population'],
                                                        w=w), name=('input', 'population'))

        # Create random inputs for `timesteps`.
        inpt = np.random.binomial(2, p_fire, size=[timesteps, n_inpt])
        inpt = torch.from_numpy(inpt)
        inpts = {'input' : inpt}
    
    def setUp(self):
        """
        Make necessary changes to the network before this particular test
        """
        pass
        
    def tearDown(self):
        """
        Reset the network to its original settings in order to not have to redefine the network for every test
        """
        pass
    
    @classmethod
    def tearDown(cls):
        """
        We could accumulate all the results in one array defined for the class and output the final results here
        as well as some other info
        """
        pass

## Example(s)

I'm working with a gauss_jordan function that is implemented and is in need of testing with different inputs. This will sufficiently demonstrate the idea behind unittest and the way of structuring the test that is useful for bigger projects. You will find "__gauss_jordan__" function inside __demo.py__. It simply takes a matrix and inverts is. If the inverse does not exist, it will return None.

In [3]:
def check_load(filename,fun_name):
    try:
        #Importing the file source code
        user_module = __import__(filename) 
    except ModuleNotFoundError: 
        print("%s FILE NOT FOUND"%filename)
        return None
    
    if hasattr(user_module,fun_name) and callable(user_module.__dict__[fun_name]):
        print("%s was succesfully loaded"%fun_name)
        return user_module.__dict__[fun_name] #return the function pointer for usage
    else: #did not find the function
        print("%s is not defined"%fun_name)
        return None

class TestGaussJordan(unittest.TestCase):
    fn = "demo"
    
    def test(self):
        print("*********** Testing gauss_jordan ***********")
        gauss_jordan = check_load(self.fn,"gauss_jordan")
        self._test1(gauss_jordan,"test 1")
        self._test2(gauss_jordan,"test 2")
        self._test_non_invertable(gauss_jordan,"test non-invertible")
    
    def _test1(self,gauss_jordan,test_name):
        print("---- %s ----"%test_name)
        A = np.array([[1,3],[2,5]])
        A_inv = np.array([[-5,3],[2,-1]])
        got = gauss_jordan(A)
        print("input:")
        print(A)
        print("expected:")
        print(A_inv)
        print("got:")
        print(got)

        if np.all(np.isclose(got,A_inv)):
            print("result: Correct\n")
        else:
            print("result: Incorrect!\n")
        
    def test4(self):
        from demo import gauss_jordan
        A = np.array([[1,3],[2,5]])
        A_inv = np.array([[-5,3],[2,-1]])
        got = gauss_jordan(A)
        print("input:")
        print(A)
        print("expected:")
        print(A_inv)
        print("got:")
        print(got)
    
        if np.all(np.isclose(got,A_inv)):
            print("result: Correct\n")
        else:
            print("result: Incorrect!\n")
            
    def _test2(self,gauss_jordan,test_name):
        print("---- %s ----"%test_name)
        A = np.array([[2,3,0],[1,-2,-1],[2,0,-1]])
        A_inv = np.array([[2,3,-3],[-1,-2,2],[4,6,-7]])
        got = gauss_jordan(A)
        print("input:")
        print(A)
        print("expected:")
        print(A_inv)
        print("got:")
        print(got)

        if np.all(np.isclose(got,A_inv)):
            print("result: Correct\n")
        else:
            print("result: Incorrect!\n")
        
    def _test_non_invertable(self,gauss_jordan,test_name):
        """
            A non-invertible example.
        """
        print("---- %s ----"%test_name)
        A = np.array([[2,3,0],[1,-2,-1],[2,-4,-2]])
        got = gauss_jordan(A)
        print("input:")
        print(A)
        print("expected:")
        print(None)
        print("got:")
        print(got)

        if got is None:
            print("result: Correct\n")
        else:
            print("result: Incorrect!\n")

Running __demo_tests.py__ will produce the  output shown below. This unittest.main seems different because unittest.main looks at sys.argv by default, which is what started IPython, hence the error about the kernel connection file not being a valid attribute. You can pass an explicit list to main to avoid looking up sys.argv. [Source](https://stackoverflow.com/questions/37895781/unable-to-run-unittests-main-function-in-ipython-jupyter-notebook). This is only a problm for jupyter. In your personal IDE, it would simply be __unittest.main()__ as before and as it is in the example code.

In [4]:

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..

*********** Testing gauss_jordan ***********
gauss_jordan was succesfully loaded
---- test 1 ----
input:
[[1 3]
 [2 5]]
expected:
[[-5  3]
 [ 2 -1]]
got:
[[-5.  3.]
 [ 2. -1.]]
result: Correct

---- test 2 ----
input:
[[ 2  3  0]
 [ 1 -2 -1]
 [ 2  0 -1]]
expected:
[[ 2  3 -3]
 [-1 -2  2]
 [ 4  6 -7]]
got:
[[ 2.  3. -3.]
 [-1. -2.  2.]
 [ 4.  6. -7.]]
result: Correct

---- test non-invertible ----
input:
[[ 2  3  0]
 [ 1 -2 -1]
 [ 2 -4 -2]]
expected:
None
got:
None
result: Correct

input:
[[1 3]
 [2 5]]
expected:
[[-5  3]
 [ 2 -1]]
got:
[[-5.  3.]
 [ 2. -1.]]
result: Correct




----------------------------------------------------------------------
Ran 2 tests in 0.034s

OK


### A quicker way to do the same type of tests

In [2]:
class TestGaussJordan(unittest.TestCase):
    def test(self):
        print("*********** Testing gauss_jordan ***********")
        from demo import gauss_jordan #importing necessary function 
        A = np.array([[1,3],[2,5]])
        A_inv = np.array([[-5,3],[2,-1]])
        got = gauss_jordan(A)
        print("input:")
        print(A)
        print("expected:")
        print(A_inv)
        print("got:")
        print(got)
    
        if np.all(np.isclose(got,A_inv)):
            print("result: Correct\n")
        else:
            print("result: Incorrect!\n")

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.

*********** Testing gauss_jordan ***********
input:
[[1 3]
 [2 5]]
expected:
[[-5  3]
 [ 2 -1]]
got:
[[-5.  3.]
 [ 2. -1.]]
result: Correct




----------------------------------------------------------------------
Ran 1 test in 0.012s

OK


This was a brief introduction to unittest followed by examples to demonstrate usage. Again, the code used here is inside __spiketorch/examples/testing/demo.py__ and __demo_tests.py__. 